# Inżynieria cech
 - badanie wartości NA, błędów grubych
 - badanie typów danych i zmniejszenie zużycia pamięci
 - stworzenie danych średnich w różnych oknach czasowych (10 ostatnich spotkań, 20, 30 i od początku sezonu)

In [ ]:
import pandas as pd
import sqlite3
import numpy as np

pd.set_option('display.max_columns', 40)

In [ ]:
con = sqlite3.connect('data/start/NBA-Game-Database-combined.sqlite')
game_info = pd.read_sql_query(f"SELECT * FROM \"{'game_info'}\" ORDER BY date", con)
team_stats = pd.read_sql_query(f"SELECT * FROM \"{'team_stats'}\"", con)
player_stats = pd.read_sql_query(f"SELECT * FROM \"{'player_stats'}\"", con)
con.close()

## Badanie wartości NA, outlierów i błędów logicznych w danych

### Dane drużynowe

In [ ]:
def analyze_na_outliers(df, context_cols):
    missing_data = pd.isnull(df).mean()*100

    # Wykrywanie grubych błędów
    outliers = {}
    for col in df.select_dtypes(include=['float', 'int']).columns:
        mean = df[col].mean()
        std = df[col].std()
        threshold = 4

        outliers[col] = df.loc[
            (df[col] < mean - threshold * std) | (df[col] > mean + threshold * std),
            context_cols + [col]
        ]

    errors = {}
    for col in df.loc[:, df.columns.str.contains('%')].columns:
        errors[col] = df.loc[
            df[col]<0,
            context_cols + [col]
        ]

    return missing_data, outliers, errors

In [ ]:
team_missing_data, team_outliers, team_errors = analyze_na_outliers(team_stats, context_cols=['game_id', 'team'])
team_missing_data[team_missing_data > 0]

In [ ]:
team_stats[team_stats['FT%'].isna()]

Kolumnę +/- można usunąć, Net Rating przekazuje tą samą informację, ale z uwzględnieniem tempa gry

In [ ]:
team_stats.loc[16476, 'FT%'] = 0.0
team_stats = team_stats.drop(columns=['+/-', 'GmSc', 'USG%'])

In [ ]:
for column, outlier_values in team_outliers.items():
    if not outlier_values.empty:
        print(outlier_values, end='\n\n')

Outliery wskazują na rzadko spotykane wydarzenia i ciekawostki statystyczne, nie widać niepoprawnych wartości

In [ ]:
for column, error_values in team_errors.items():
    if not error_values.empty:
        print(error_values, end='\n\n')

Brak wychwyconych błędów logicznych w danych drużynowych

### Dane wg zawodników

In [ ]:
player_missing_data, player_outliers, player_errors = analyze_na_outliers(player_stats, context_cols=['game_id', 'player'])
player_missing_data

Nie będziemy imputować danych w tabeli player_stats, ponieważ kolumny z brakami nie będą wykorzystywane.

In [ ]:
for column, outlier_values in player_outliers.items():
    if not outlier_values.empty:
        print(outlier_values, end='\n\n')

Po analizie outlierów można stwierdzić, że część wartości to rekordowe, rzadko spotykane liczby - tyczy się to wartości nominalnych.

Outliery w kolumnach procentowych (druga połowa outputu, np. TRB% - procent wszystkich piłek możliwych do zebrania zebranych przez danego zawodnika) są często spowodowane krótkim czasem gry zawodników, co przełożyło się na nienaturalnie duże wartości

In [ ]:
for column, error_values in player_errors.items():
    if not error_values.empty:
        print(error_values, end='\n\n')

AST% jest definiowana jako szacunkowa wartość procentowa trafionych rzutów przez kolegów z drużyny, przy których asystował zawodnik, gdy był on na boisku. Nie może zatem ona być mniejsza niż 0, więc zastąpimy te wartości

In [ ]:
player_stats.loc[player_stats['AST%']<0, 'AST%'] = 0.0

## Badanie typów danych i zmniejszenie zużycia pamięci

### Team stats

In [ ]:
team_stats.dtypes

In [ ]:
team_stats.memory_usage(deep=True)

In [ ]:
team_stats.memory_usage(deep=True).sum()

In [ ]:
(team_stats
.select_dtypes(int)
.describe
)

In [ ]:
team_stats = team_stats.astype({'win': 'int8'})

### Player stats

In [ ]:
player_stats.dtypes

In [ ]:
player_stats.memory_usage(deep=True)

In [ ]:
player_stats.memory_usage(deep=True).sum()

In [ ]:
player_stats.columns

In [ ]:
player_stats = player_stats.drop(
    columns=['FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']
)

W kolumnie Minutes Played jest wyszczególniona informacja o przyczynie nieobecności zawodnika. Nie jest ona potrzebna w modelowaniu

In [ ]:
mp_values = ['Did Not Play', 'Player Suspended', 'Not With Team', 'Did Not Dress']
player_stats.loc[player_stats['MP'].isin(mp_values), 'MP'] = np.nan

In [ ]:
player_stats.memory_usage(deep=True)

In [ ]:
player_stats.memory_usage(deep=True).sum()

### Dodanie daty

In [ ]:
def extract_date(game_id):
    game_id_str=str(game_id)
    season_prefix = game_id_str[2:4]
    month = int(game_id_str[4:6])
    day = int(game_id_str[6:8])

    if 10 <= month <= 12:
        year = '20'+str(int(season_prefix) - 1)
    else:
        year = '20'+str(int(season_prefix))

    date_str = f"{year}-{month:02d}-{day:02d}"
    return date_str

In [ ]:
team_stats['Date'] = team_stats['game_id'].apply(extract_date)
team_stats['Date']

### Dodanie kolumn z liczbą zwycięstw w ostatnich 10 spotkaniach i aktualną serią porażek/zwycięstw

In [ ]:
team_stats['Season'] = team_stats['game_id'].astype(str).str[:4]
# team_stats['Season']

In [ ]:
team_stats['id'] = team_stats.index

In [ ]:
def add_last_10(df):
    df['last10'] = (
        df.groupby(['team', 'Season'])['win']
        .rolling(window=10, min_periods=1)
        .sum()
        .shift(1)
        .reset_index(level=['team', 'Season'], drop=True)
    )

    first_game_mask = df.groupby(['team', 'Season']).cumcount() == 0
    df.loc[first_game_mask, 'last10'] = 0.0
    return df.sort_index()

In [ ]:
def calculate_streak(win_series):
    streaks = []
    streak = 0
    last_win = None

    for i in range(1, len(win_series)):
        win = win_series.iloc[i - 1]  # bierzemy wynik meczu poprzedzającego aktualny
        if win == last_win:
            streak += 1
        else:
            streak = 1
        streaks.append(streak if win == 1 else -streak)
        last_win = win

    # Pierwszy mecz nie może mieć streak, więc dodajemy wartość 0 na początku
    streaks.insert(0, 0)

    return streaks


In [ ]:
team_stats = add_last_10(team_stats)
team_stats

In [ ]:
team_stats['streak'] = team_stats \
    .groupby(['team', 'Season'])['win'] \
    .transform(calculate_streak)
team_stats = team_stats.sort_index()
team_stats[team_stats['team']=='GSW']

### Stworzenie ramek danych w oknach czasowych

In [ ]:
def calculate_rolling_stats(df, window, columns, cols_concat):
    df_copy = df.copy()
    results = pd.DataFrame()

    for col in columns:
        if window == 'all':
            # Calculate the mean for each team-season group up to the current game (shifted by one)
            rolling_mean = (
                df_copy.groupby(['Season', 'team'])[col]
                .expanding()
                .mean()
                .shift(1)
                .reset_index(level=['Season', 'team'], drop=True)
            )
        else:
            # Calculate rolling mean with a specified window
            rolling_mean = (
                df_copy.groupby(['Season', 'team'])[col]
                .rolling(window=window, min_periods=1)
                .mean()
                .shift(1)
                .reset_index(level=['Season', 'team'], drop=True)
            )

        results[col] = rolling_mean

    return pd.concat([df_copy[cols_concat].reset_index(drop=True), results], axis=1)

In [ ]:
team_stats.columns

In [ ]:
cols_move = ['id', 'game_id', 'Date', 'Season', 'team', 'win',  'streak', 'last10'] # kolumny do przeniesienia do docelowego df
cols_to_transform = team_stats.columns[2:-7]

In [ ]:
team_all_season = calculate_rolling_stats(team_stats, 'all', cols_to_transform, cols_move)
team_all_season[team_all_season['team']=='BOS']

Należy poprawić manualnie pierwsze rekordy w sezonie - średnie kroczące jak i last10. Przez przesunięcie shift() pierwsze rekordy są wzięte z poprzedniej drużyny, a pierwsza drużyna wg alfabetu - ATL (Atlanta Hawks) posiada wartości NAN

W pierwszym sezonie w ramce danych każda drużyna będzie miała wpisane średnie dla całej ligi z całego sezonu, a w każdym następnym pierwszym rekordem będzie średnia dla drużyny z poprzedniego sezonu. Imputowany pierwszy rekord nie będzie mieć wpływu na dalsze obliczenia średnich kroczących

In [ ]:
def impute_first_rows(rolling_avgs, team_stats_boxscore, columns):
    first_season = '1718'
    df_copy = team_stats_boxscore.copy()
    rolling_avgs = rolling_avgs.copy()

    # Dla każdego sezonu (oprócz pierwszego) używamy ostatnich wartości z poprzedniego sezonu
    for season in df_copy['Season'].unique()[1:]:  # pomijamy pierwszy sezon
        yr1, yr2 = int(season[:2]), int(season[2:])
        prev_season = str(yr1-1)+str(yr2-1)  # poprzedni sezon

        # Dla każdej drużyny w bieżącym sezonie
        for team in np.sort(df_copy[df_copy['Season'] == season]['team'].unique()):
            # Znajdujemy indeks pierwszego rekordu w bieżącym sezonie
            current_mask = (rolling_avgs['Season'] == season) & (rolling_avgs['team'] == team)

            # Znajdujemy ostatnią wartość z poprzedniego sezonu
            prev_mask = (rolling_avgs['Season'] == prev_season) & (rolling_avgs['team'] == team)

            if prev_mask.any():
                prev_values = rolling_avgs[prev_mask].iloc[-1][columns]
                if current_mask.any():
                    first_idx = rolling_avgs[current_mask].index[0]
                    rolling_avgs.loc[first_idx, columns] = prev_values

    # Dla pierwszego sezonu, używamy średniej ligowej
    league_avg = df_copy[df_copy['Season'] == first_season][columns].mean()

    for team in np.sort(df_copy[df_copy['Season'] == first_season]['team'].unique()):
        mask = (rolling_avgs['Season'] == first_season) & (rolling_avgs['team'] == team)
        if mask.any():
            first_idx = rolling_avgs[mask].index[0]
            rolling_avgs.loc[first_idx, columns] = league_avg

    return rolling_avgs

In [ ]:
team_all_season = impute_first_rows(team_all_season, team_stats, cols_to_transform)
team_all_season[team_all_season['team']=='BOS']

In [ ]:
team_all_season[(team_all_season['team']=='HOU')]

Teraz możemy stworzyć także ramki dla okien czasowych 20, 30 i 40 meczowych

In [ ]:
team_last_20 = calculate_rolling_stats(team_stats, 20, cols_to_transform, cols_move)
team_last_20 = impute_first_rows(team_all_season, team_stats, cols_to_transform)
team_last_20

In [ ]:
team_last_30 = calculate_rolling_stats(team_stats, 30, cols_to_transform, cols_move)
team_last_30 = impute_first_rows(team_all_season, team_stats, cols_to_transform)
team_last_30

In [ ]:
team_last_40 = calculate_rolling_stats(team_stats, 40, cols_to_transform, cols_move)
team_last_40 = impute_first_rows(team_all_season, team_stats, cols_to_transform)
team_last_40

In [ ]:
any(team_last_40.isna().sum())

In [ ]:
team_last_30.isna().sum()

In [ ]:
team_last_20.isna().sum()

In [ ]:
team_all_season.isna().sum()

In [ ]:
with sqlite3.connect('data/transformed/team_moving_avgs.sqlite') as con:
    team_last_20.to_sql('team_last_20', con, if_exists='replace', index=False)
    team_last_30.to_sql('team_last_30', con, if_exists='replace', index=False)
    team_last_40.to_sql('team_last_40', con, if_exists='replace', index=False)
    team_all_season.to_sql('team_all_season', con, if_exists='replace', index=False)